In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#LOAD THE DATA
from google.colab import files ;upload = files.upload()

Saving Material List-MARA.XLSX to Material List-MARA.XLSX


In [ ]:
import pandas as pd

# Replace 'your_file.xlsx' with the actual path to your Excel file
file_path = 'Material List-MARA.XLSX'

# Read Excel file into a DataFrame
data = pd.read_excel('Material List-MARA.XLSX')

# Now 'data' contains your Excel data


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load your dataset (replace 'your_data.csv' with the actual file path)
df = pd.read_excel('Material List-MARA.XLSX')

# Choose a pre-trained model
model = SentenceTransformer('stsb-roberta-large')

# Clean NaN values in 'Material Description' column
df['Material Description'] = df['Material Description'].fillna('')

# Encode material descriptions to obtain their vector representations
df['material_description_embedding'] = df['Material Description'].apply(
    lambda x: model.encode(x, convert_to_tensor=True) if isinstance(x, str) else torch.zeros(model.get_sentence_embedding_dimension())
)

# Set a similarity threshold (adjust as needed)
threshold = 0.8

# Function to identify duplicates based on similarity
def find_duplicates(row):
    if torch.all(row['material_description_embedding'] == 0.0):
        # Handle cases where the embedding is empty (all zeros)
        return []

    # Convert tensors to lists
    embedding_row = row['material_description_embedding'].tolist()
    embeddings_df = df['material_description_embedding'].apply(lambda x: x.tolist()).tolist()

    similarities = util.pytorch_cos_sim(torch.tensor(embedding_row), torch.tensor(embeddings_df)).flatten()
    duplicate_indices = [i for i, sim in enumerate(similarities) if sim > threshold and i != row.name]
    return df.iloc[duplicate_indices]['Material Description'].tolist()

# Apply the function to each row to find duplicates
df['potential_duplicates'] = df.apply(find_duplicates, axis=1)

# Display potential duplicates
print(df[['Material Description', 'potential_duplicates']])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

     Material Description                               potential_duplicates
0       NPI Test Material  [test npi, Test NPI, NPI Test Material, NPI Te...
1    TEMPLATE_MAT_FERT_01                                                 []
2    TEMPLATE_MAT_FERT_02                                                 []
3    TEMPLATE_MAT_FERT_09                                                 []
4           Test material                                                 []
..                    ...                                                ...
878                Theeru                        [Created By theeru, Theeru]
879                Theeru                        [Created By theeru, Theeru]
880        Test by theeru  [Test by Theeru, Test by Theeru, Test by Theer...
881        Test by theeru  [Test by Theeru, Test by Theeru, Test by Theer...
882        Test by theeru  [Test by Theeru, Test by Theeru, Test by Theer...

[883 rows x 2 columns]


In [ ]:
# Apply the function to each row to find duplicates
df['potential_duplicates'] = df.apply(find_duplicates, axis=1)

# Count the number of potential duplicates for each material
df['num_duplicates'] = df['potential_duplicates'].apply(len)

# Display the number of duplicates for each material
print(df[['Material Description', 'num_duplicates']])

     Material Description  num_duplicates
0       NPI Test Material               6
1    TEMPLATE_MAT_FERT_01               0
2    TEMPLATE_MAT_FERT_02               0
3    TEMPLATE_MAT_FERT_09               0
4           Test material               0
..                    ...             ...
878                Theeru               2
879                Theeru               2
880        Test by theeru               9
881        Test by theeru               9
882        Test by theeru               9

[883 rows x 2 columns]


In [ ]:
import pandas as pd

# Count occurrences of each material description
duplicates = df.groupby('Material Description').size().reset_index(name='Num Duplicates')

# Filter out materials with only one occurrence (no duplicates)
duplicates = duplicates[duplicates['Num Duplicates'] > 1]

# Display duplicates for each material description
print(duplicates)


                        Material Description  Num Duplicates
0                                                          2
1                                          .               3
7                                  Aaquarium               2
10                                Air Filter               2
25                                  BB Cream               2
33                                   Bearing               2
42                                     Bolt1               2
89                                  DIO Gear               2
122                           Eyebrow pencil               2
124                                Eyeshadow               2
136                FIN126,MTS-DI,PD,SerialNo               2
146                              Face Powder               2
165                                 Forklift               2
193                                 Head Set               2
199                      INC-0001-E1-HE-Bolt               3
202                     

In [ ]:
import pandas as pd

# Filter rows with material description 'Test by Theeru'
duplicates = df[df['Material Description'] == 'Test by Theeru']

# Display the duplicates
print(duplicates)


    Material Created On Created At Time Created By Last Change Changed By  \
361      874 2023-12-20        15:25:23   INC02354  2023-12-20   INC02354   
362      876 2023-12-21        08:58:11   INC02354  2023-12-21   INC02354   
841  TEST_M1 2023-12-21        08:23:44   INC02354  2023-12-21   INC02354   
865     ZSN5 2023-12-21        09:50:14   INC02354  2023-12-21   INC02354   
866     ZSN6 2023-12-21        10:10:22   INC02354  2023-12-21   INC02354   
871   ZSNT32 2024-01-03        14:23:55   INC02354  2024-01-03   INC02354   
875   ZSNTM4 2023-12-20        14:47:27   INC02354  2023-12-20   INC02354   

    Compl. maint. status Maintenance status DF at client level Material type  \
361                   KB                 KB                NaN           MAT   
362                    K                  K                NaN           MAT   
841                    K                  K                NaN           MAT   
865                    K                  K                NaN 